In [128]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import xgboost as xgb
import warnings
import matplotlib.pyplot as plt
import plotly.express as px



In [2]:
path_to_data = r"C:\Users\meghn\Documents\Dauphine\Master\M2 Finance Quantitative\Semestre 2\Machine Learning\data\\"
train_home_team_statistics_df = pd.read_csv(path_to_data+'Train_Data/train_home_team_statistics_df.csv', index_col=0)
train_away_team_statistics_df = pd.read_csv(path_to_data+'Train_Data/train_home_team_statistics_df.csv', index_col=0)
train_scores = pd.read_csv(path_to_data+'Y_train_1rknArQ.csv', index_col=0)

train_home = train_home_team_statistics_df.iloc[:,2:]
train_away = train_away_team_statistics_df.iloc[:,2:]

train_home.columns = 'HOME_' + train_home.columns
train_away.columns = 'AWAY_' + train_away.columns

train_data =  pd.concat([train_home,train_away],join='inner',axis=1)
train_scores = train_scores.loc[train_data.index]
train_data = train_data.replace({np.inf:np.nan,-np.inf:np.nan})

In [5]:
train_df = train_data.reset_index().merge(train_scores.reset_index(), on="ID").set_index('ID')
corr_matrix = train_df.corr().select_dtypes(include=[float, int])

#
# target_correlations_clean = corr_matrix_numeric[target_names].stack().reset_index()
# target_correlations_clean.columns = ['Feature', 'Target', 'Correlation']



,HOME_TEAM_SHOTS_TOTAL_season_sum,HOME_TEAM_SHOTS_INSIDEBOX_season_sum,HOME_TEAM_SHOTS_OFF_TARGET_season_sum,HOME_TEAM_SHOTS_ON_TARGET_season_sum,HOME_TEAM_SHOTS_OUTSIDEBOX_season_sum,HOME_TEAM_PASSES_season_sum,HOME_TEAM_SUCCESSFUL_PASSES_season_sum,HOME_TEAM_SAVES_season_sum,HOME_TEAM_CORNERS_season_sum,HOME_TEAM_FOULS_season_sum,...,AWAY_TEAM_ATTACKS_5_last_match_std,AWAY_TEAM_PENALTIES_5_last_match_std,AWAY_TEAM_SUBSTITUTIONS_5_last_match_std,AWAY_TEAM_BALL_SAFE_5_last_match_std,AWAY_TEAM_DANGEROUS_ATTACKS_5_last_match_std,AWAY_TEAM_INJURIES_5_last_match_std,AWAY_TEAM_GOALS_5_last_match_std,HOME_WINS,DRAW,AWAY_WINS
HOME_TEAM_SHOTS_TOTAL_season_sum,1.000000,0.847872,0.754090,0.768145,0.658873,0.507362,0.503568,-0.238034,0.608980,-0.026233,...,0.085557,0.104877,-0.012672,0.040317,0.155491,-0.044459,0.200021,0.169132,-0.042921,-0.141299
HOME_TEAM_SHOTS_INSIDEBOX_season_sum,0.847872,1.000000,0.667136,0.756885,0.345126,0.532697,0.518325,-0.287633,0.619344,-0.059692,...,0.077454,0.118955,0.014335,0.036894,0.161847,-0.053919,0.227214,0.192106,-0.055714,-0.153633
HOME_TEAM_SHOTS_OFF_TARGET_season_sum,0.754090,0.667136,1.000000,0.530674,0.581112,0.320187,0.316511,-0.217051,0.510795,0.024388,...,0.059048,0.047628,0.008799,0.014817,0.132476,-0.021225,0.105399,0.132605,-0.036037,-0.108442
HOME_TEAM_SHOTS_ON_TARGET_season_sum,0.768145,0.756885,0.530674,1.000000,0.456791,0.538341,0.536642,-0.223675,0.579676,-0.102117,...,0.089840,0.129428,0.019101,0.025737,0.142130,-0.040861,0.269905,0.175693,-0.051992,-0.139651
HOME_TEAM_SHOTS_OUTSIDEBOX_season_sum,0.658873,0.345126,0.581112,0.456791,1.000000,0.311601,0.320214,-0.152272,0.346805,-0.053134,...,0.071975,0.017403,0.001982,0.005130,0.116312,0.009037,0.089760,0.071879,-0.025516,-0.053051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AWAY_TEAM_INJURIES_5_last_match_std,-0.044459,-0.053919,-0.021225,-0.040861,0.009037,-0.072902,-0.074101,-0.018287,-0.035382,0.057286,...,-0.014823,0.007571,-0.031049,0.021602,-0.011114,1.000000,-0.030204,-0.023054,0.011201,0.014260
AWAY_TEAM_GOALS_5_last_match_std,0.200021,0.227214,0.105399,0.269905,0.089760,0.178054,0.178654,-0.091953,0.168503,-0.063134,...,0.044330,0.118746,0.012652,0.006922,0.041487,-0.030204,1.000000,0.079928,-0.029356,-0.058097
HOME_WINS,0.169132,0.192106,0.132605,0.175693,0.071879,0.162543,0.168345,-0.098000,0.157698,-0.057202,...,0.029500,0.038268,0.007002,0.005047,0.045399,-0.023054,0.079928,1.000000,-0.519061,-0.582631
DRAW,-0.042921,-0.055714,-0.036037,-0.051992,-0.025516,-0.050942,-0.052100,0.007525,-0.038361,0.020647,...,-0.005833,-0.009253,-0.011263,-0.001205,-0.007484,0.011201,-0.029356,-0.519061,1.000000,-0.392255


In [6]:
corr_matrix.to_excel("correlation_matrix.xlsx")

In [9]:
target_correlations = corr_matrix[['HOME_WINS', 'DRAW', 'AWAY_WINS']].copy()

In [133]:
targets = ['HOME_WINS', 'DRAW', 'AWAY_WINS'] ## Group home wins or away winsn and then do another model where we predict whether it's a draw. 1) predict draw 2) predict win or not
important_corr_threshold = 0.075  # Threshold for considering a feature important
high_corr_threshold = 0.7    # Threshold for considering features redundant
important_features_by_target = {}
for target in targets:
    important_features = corr_matrix[target][corr_matrix[target].abs() > important_corr_threshold].index.tolist()
    important_features_by_target[target] = [feat for feat in important_features if feat not in targets]

In [134]:
final_features_to_keep = {}
for target, features in important_features_by_target.items():
    non_redundant_features = []
    checked = set()
    for feature in features:
        if feature not in checked:
            highly_correlated = corr_matrix.loc[features, feature].abs() > high_corr_threshold
            correlated_features = highly_correlated.index[highly_correlated].tolist()
            if correlated_features:
                best_feature = max(correlated_features, key=lambda x: abs(corr_matrix.at[x, target]))
                non_redundant_features.append(best_feature)
                checked.update(correlated_features)
            else:
                non_redundant_features.append(feature)
                checked.add(feature)
    final_features_to_keep[target] = non_redundant_features


In [135]:
print("Important Features by Target:", important_features_by_target)
print("Final Features to Keep:", final_features_to_keep)
all_features = set()
for features in final_features_to_keep.values():
    all_features.update(features)


Important Features by Target: {'HOME_WINS': ['HOME_TEAM_SHOTS_TOTAL_season_sum', 'HOME_TEAM_SHOTS_INSIDEBOX_season_sum', 'HOME_TEAM_SHOTS_OFF_TARGET_season_sum', 'HOME_TEAM_SHOTS_ON_TARGET_season_sum', 'HOME_TEAM_PASSES_season_sum', 'HOME_TEAM_SUCCESSFUL_PASSES_season_sum', 'HOME_TEAM_SAVES_season_sum', 'HOME_TEAM_CORNERS_season_sum', 'HOME_TEAM_ATTACKS_season_sum', 'HOME_TEAM_DANGEROUS_ATTACKS_season_sum', 'HOME_TEAM_GOALS_season_sum', 'HOME_TEAM_GAME_WON_season_sum', 'HOME_TEAM_GAME_LOST_season_sum', 'HOME_TEAM_SHOTS_TOTAL_season_average', 'HOME_TEAM_SHOTS_INSIDEBOX_season_average', 'HOME_TEAM_SHOTS_OFF_TARGET_season_average', 'HOME_TEAM_SHOTS_ON_TARGET_season_average', 'HOME_TEAM_PASSES_season_average', 'HOME_TEAM_SUCCESSFUL_PASSES_season_average', 'HOME_TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average', 'HOME_TEAM_SAVES_season_average', 'HOME_TEAM_CORNERS_season_average', 'HOME_TEAM_BALL_POSSESSION_season_average', 'HOME_TEAM_ATTACKS_season_average', 'HOME_TEAM_DANGEROUS_ATTACKS_se

In [136]:
filtered_data = train_data[list(all_features)]
categorical_features = filtered_data.select_dtypes(include=['object']).columns.tolist()
numerical_features = filtered_data.select_dtypes(include=[np.number]).columns.tolist()

In [137]:
from sklearn.base import BaseEstimator, ClassifierMixin

class DrawClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, classifier):
        self.classifier = classifier

    def fit(self, X, y):
        y_draw = (y == 'DRAW').astype(int)
        self.classifier.fit(X, y_draw)
        return self

    def predict(self, X):
        return self.classifier.predict(X)

    def predict_proba(self, X):
        return self.classifier.predict_proba(X)

from sklearn.linear_model import LogisticRegression
draw_classifier = DrawClassifier(LogisticRegression())


In [125]:
nested_score

array([0.16532553, 0.17751768, 0.18166301])

In [139]:
class WinLossClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, classifier):
        self.classifier = classifier

    def fit(self, X, y):
        # Filter out draws and train on win/loss outcomes
        non_draw_indices = y != 'DRAW'
        self.classifier.fit(X[non_draw_indices], y[non_draw_indices])
        return self

    def predict(self, X):
        return self.classifier.predict(X)

    def predict_proba(self, X):
        return self.classifier.predict_proba(X)

# Example classifier for win/loss
win_loss_classifier = WinLossClassifier(LogisticRegression())


In [140]:
class StagedPredictionModel(BaseEstimator, ClassifierMixin):
    def __init__(self, draw_model, win_loss_model):
        self.draw_model = draw_model
        self.win_loss_model = win_loss_model

    def fit(self, X, y):
        # First, train the draw classifier
        self.draw_model.fit(X, y)
        # Then, train the win/loss classifier
        self.win_loss_model.fit(X, y)
        return self

    def predict(self, X):
        # First, predict draws
        draw_preds = self.draw_model.predict(X)
        # Where not draw, predict win/loss
        win_loss_preds = self.win_loss_model.predict(X)
        # Combine predictions
        return np.where(draw_preds == 1, 'Draw', win_loss_preds)

staged_model = StagedPredictionModel(draw_classifier, win_loss_classifier)

final_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', staged_model)
])


In [143]:
final_pipeline.fit(X, y)
predictions = final_pipeline.predict(X)

# Evaluate the model
from sklearn.metrics import classification_report
print(classification_report(y, predictions))


ValueError: y should be a 1d array, got an array of shape (12303, 3) instead.